In [42]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

## nn.Module
* Base class for all neural network modules
* 只要在nn.Module的子類中定義了forward函數，backward函數就會被自動實現（利用Autograd）
* nn.Conv2d 本身也是nn.Module的類別(此時我們可以先不用理解nn.Conv2D做了什麼，只需了解其包含一些參數與操作)

In [43]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5)
        self.conv2 = nn.Conv2d(20, 20, 5)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        return F.relu(self.conv2(x))

In [44]:
model = Model()

### 實踐 forward propagation 
* 為什麼不應該直接call model.forward : https://discuss.pytorch.org/t/any-different-between-model-input-and-model-forward-input/3690

In [45]:
input_ = torch.randn(1,1,124,124)
output = model(input_)

### 查看 model 底下的 modules

#### .modules

* model.modules 遞迴的列出所有的 modules

In [46]:
for module in model.modules():
    print(module)

Model(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 20, kernel_size=(5, 5), stride=(1, 1))
)
Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
Conv2d(20, 20, kernel_size=(5, 5), stride=(1, 1))


#### .children

* model.children 只列出第一層的子 modules

In [47]:
for module in model.children():
    print(module)

Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
Conv2d(20, 20, kernel_size=(5, 5), stride=(1, 1))


### 查看 model 內的 parameters (torch.nn.parameter.Parameter)

#### .named_parameters
* named_parameters會列出每個nn.Module底下parameters 的名字,數值
* 同時可以查看 requires_grad是否開啟(for backpropagation)

In [48]:
for name, param in model.named_parameters():
    print(name,param.requires_grad)
    #param.requires_grad=True

conv1.weight True
conv1.bias True
conv2.weight True
conv2.bias True


#### .parameters
* 不會印出名字

In [49]:
for param in model.parameters():
    print(type(param),param.shape, param.requires_grad)

<class 'torch.nn.parameter.Parameter'> torch.Size([20, 1, 5, 5]) True
<class 'torch.nn.parameter.Parameter'> torch.Size([20]) True
<class 'torch.nn.parameter.Parameter'> torch.Size([20, 20, 5, 5]) True
<class 'torch.nn.parameter.Parameter'> torch.Size([20]) True


#### 計算模型可訓練參數總量

In [50]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print('總共參數量：' ,params)

總共參數量： 10540


### Backpropagation

In [51]:
input_ = torch.randn(1,1,124,124)
output = model(input_)

#### 確認 requires_grad為 True (default 就是 True)

In [52]:
for name, param in model.named_parameters():
    print(name,param.requires_grad)

conv1.weight True
conv1.bias True
conv2.weight True
conv2.bias True


#### 此時還沒做backpropagation，parameters沒有gradient value

In [53]:
print(model.conv1.weight.grad)

None


#### 執行backward，完成後就能看到每個parameters底下的gradient value

In [55]:
output.sum().backward()

In [153]:
print(model.conv1.weight.grad)

tensor([[[[-3.9841e+02,  6.0265e+01,  8.3941e+02, -4.0811e+02, -5.0551e+02],
          [-4.5694e+01,  1.9287e+02,  6.8446e+02,  7.0936e+02, -4.9671e+02],
          [ 1.2062e+02, -4.9557e+02,  2.0279e+02,  1.7787e+00,  2.4515e+02],
          [-3.5738e+02, -6.4618e+02,  3.5662e+02, -3.4831e+02, -2.6697e+02],
          [-3.5568e+02,  5.9962e+02, -2.4759e+02, -7.1152e+02,  7.8287e+02]]],


        [[[-3.2128e+02, -3.4191e+01,  1.0238e+02, -4.2600e+02,  1.0047e+01],
          [-2.0660e+02, -1.1541e+02,  1.8130e+02,  2.6142e+02, -3.5687e+02],
          [ 3.4553e+02, -2.0279e+02,  3.9684e+02,  1.5776e+02,  6.0539e+02],
          [ 4.3211e+02,  3.0171e+02, -1.8974e+02, -6.9497e+02, -4.6586e+02],
          [ 1.3787e+02,  1.7408e+02, -5.6302e+02, -2.7836e+02,  4.5642e+02]]],


        [[[ 4.4504e+01,  3.9325e+01, -3.1536e+01, -2.3886e+01, -4.0643e+02],
          [-3.9561e+02, -1.3097e+02, -1.9010e+02,  4.3633e+02, -4.3261e+02],
          [ 2.1376e+02, -1.9404e+02,  4.1607e+02,  1.9061e+01, -1.96

#### 當我們把 parameters 的 requires_grad關閉時，就無法成功的完成backward
* 什麼時候會關閉requires_grad關閉時？ prediction (inference)的階段
* 設定 requires_grad = True 是為了之後要做 backpropagation，在計算每個paramters的 gradient時，我們在forward propagation時需要保留額外的訊息(根據chain rule)，這會導致記憶體使用量上升與計算速度下降，然而只有在 training 階段時我們材需要做backpropagation，在 prediction (inference)的階段，我們則可以設定 requires_grad = False 來提升速度與降低記憶體使用量 

In [163]:
for param in model.parameters():
    param.requires_grad = False

In [164]:
input_ = torch.randn(1,1,124,124)
output = model(input_)

In [165]:
output.sum().backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

#### with torch.no_grad()
* 此行底下的requires_grad都會關閉

In [184]:
for param in model.parameters():
    param.requires_grad = True
with torch.no_grad():
    input_ = torch.randn(1,1,124,124)
    output = model(input_)
    output.sum().backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

### 讓我們自行搭建一個 nn.Module 並試算gradient

In [38]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.x = torch.nn.Parameter(torch.tensor(2.4,dtype=torch.float32))
        self.y = torch.nn.Parameter(torch.tensor(4.3,dtype=torch.float32))

    def forward(self, x):
        output = x*self.x**2 + x*self.y + x # 可以看成 output = w*x*x + w*y+2
        return output

In [41]:
model = Model()
input_ = torch.tensor(1.3, dtype = torch.float32)
output = model(input_)
output.backward()
# output 對 self.x 的偏微分為 2 * w * x = 2 * 1.3 * 2.4 = 6.24 
print('self.x 的 gradient : {}'.format(model.x.grad))
# output 對 self.y 的偏微分為 w = 1.3
print('self.y 的 gradient : {}'.format(model.y.grad))

self.x 的 gradient : 6.240000247955322
self.y 的 gradient : 1.2999999523162842


## Sequential
* nn.Module 的容器

In [187]:
layer = nn.Sequential(
                        nn.Conv2d(3,
                                  20,
                                  kernel_size=3,
                                  stride=1,
                                  padding=1,
                                  bias=False), 
                        nn.BatchNorm2d(20),
                        nn.LeakyReLU(inplace=True))

In [189]:
for name, param in layer.named_parameters():
    print(name,param.requires_grad)
    #param.requires_grad=True

0.weight True
1.weight True
1.bias True


In [192]:
input_ = torch.randn(1, 3, 124, 124)
output = layer(input_)

#### OrderedDict+Sequential, 讓我們替每一個module命名

In [194]:
from collections import OrderedDict

In [211]:
layer = nn.Sequential(OrderedDict([
          ('conv1', nn.Conv2d(1,20,5)),
          ('relu1', nn.ReLU()),
          ('conv2', nn.Conv2d(20,64,5)),
          ('relu2', nn.ReLU())
        ]))


In [212]:
for module in layer.modules():
    print(module)
    #param.requires_grad=True

Sequential(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (relu1): ReLU()
  (conv2): Conv2d(20, 64, kernel_size=(5, 5), stride=(1, 1))
  (relu2): ReLU()
)
Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
ReLU()
Conv2d(20, 64, kernel_size=(5, 5), stride=(1, 1))
ReLU()


In [213]:
for name, param in layer.named_parameters():
    print(name,param.requires_grad)
    #param.requires_grad=True

conv1.weight True
conv1.bias True
conv2.weight True
conv2.bias True


In [223]:
input_ = torch.randn(1, 1, 124, 124)
output = layer(input_)
print(output.shape)

torch.Size([1, 64, 116, 116])


#### append 新的 module到 sequential上

In [215]:
import torch.nn as nn

modules = []
modules.append(nn.Conv2d(1,20,5))
modules.append(nn.ReLU())
modules.append(nn.Conv2d(20,64,5))
modules.append(nn.ReLU())

layer = nn.Sequential(*modules)

In [216]:
layer

Sequential(
  (0): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU()
  (2): Conv2d(20, 64, kernel_size=(5, 5), stride=(1, 1))
  (3): ReLU()
)

In [222]:
input_ = torch.randn(1, 1, 124, 124)
output = layer(input_)
print(output.shape)

torch.Size([1, 64, 116, 116])


* 另一種方式

In [218]:
layer = torch.nn.Sequential()
layer.add_module("conv1", nn.Conv2d(1,20,5))
layer.add_module("relu1", nn.ReLU())
layer.add_module("conv2", nn.Conv2d(20,64,5))
layer.add_module("relu2", nn.ReLU())

In [219]:
layer

Sequential(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (relu1): ReLU()
  (conv2): Conv2d(20, 64, kernel_size=(5, 5), stride=(1, 1))
  (relu2): ReLU()
)

In [221]:
input_ = torch.randn(1, 1, 124, 124)
output = layer(input_)
print(output.shape)

torch.Size([1, 64, 116, 116])


## ModuleList
* 操作就像是python list, 但其內的module, parameters是可以被追蹤的

In [56]:
layer = nn.ModuleList()
layer.append(nn.Conv2d(1,20,5))
layer.append(nn.ReLU())
layer.append(nn.Conv2d(20,64,5))
layer.append(nn.ReLU())

ModuleList(
  (0): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU()
  (2): Conv2d(20, 64, kernel_size=(5, 5), stride=(1, 1))
  (3): ReLU()
)

In [57]:
input_ = torch.randn(1, 1, 124, 124)
for _, module in enumerate(layer):
    if _ == 0:
        output = module(input_)
    else:
        output = module(output)
print(output.shape)

torch.Size([1, 64, 116, 116])


* 可以追蹤是什麼意思？ nn.Module有辦法去獲取ModuleList裡面的資訊

In [250]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.layer = nn.ModuleList()
        self.layer.append(nn.Conv2d(1,20,5))
        self.layer.append(nn.ReLU())
        self.layer.append(nn.Conv2d(20,64,5))
        self.layer.append(nn.ReLU())

    def forward(self, x):
        for module in self.layer:
            x = module(x)
        return x

In [251]:
model = Model()

In [252]:
model

Model(
  (layer): ModuleList(
    (0): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(20, 64, kernel_size=(5, 5), stride=(1, 1))
    (3): ReLU()
  )
)

In [253]:
input_ = torch.randn(1, 1, 124, 124)
output = model(input_)
print(output.shape)

torch.Size([1, 64, 116, 116])


* 如果是一般的 python list

In [254]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.layer = []
        self.layer.append(nn.Conv2d(1,20,5))
        self.layer.append(nn.ReLU())
        self.layer.append(nn.Conv2d(20,64,5))
        self.layer.append(nn.ReLU())

    def forward(self, x):
        for module in self.layer:
            x = module(x)
        return

In [255]:
model = Model()

In [256]:
model

Model()